In [1]:
from sac import *
import model
import gym_reversi
import gym
import torch
import numpy as np
import random
from replay_memory import ReplayMemory

def opponent(board, avail):
    dummy=np.where(avail==np.amax(avail))
    # dummy=np.where(avail>0)
    maxavail = list(zip(dummy[0],dummy[1]))
    move = random.choice(maxavail)    
    return move

env = gym.make("reversi-v0", opponent = opponent, AI_Player = 1)

device = torch.device("cpu")

st_1, _,_,_ = env.step(19)

st = np.array([env.reset()])
st_torch = torch.FloatTensor(st).to(device)
memory = ReplayMemory(1000000)

In [2]:
env.reset()
cur_state = env.get_state()
valid_actions = env.get_actions(env.AI_Player)

action = 19

next_state, reward, done, _ = env.step(action)
valid_actions_next = env.get_actions(env.AI_Player)


memory.push(cur_state, action, reward, next_state, done, valid_actions, valid_actions_next)

In [3]:
# memory.sample(1)

In [4]:
agent = SAC()
# print("1: ", agent.select_action(cur_state, valid_actions))


agent.update_parameters(memory, 1,1)



RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

[(array([[[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0

In [8]:
st_torch
v = model.ValueNetwork().to(device)
sum(p.numel() for p in v.parameters()) # Same as Keras!
v.forward(st_torch)



tensor([0.0125], grad_fn=<AddBackward0>)

In [9]:
q = model.QNetwork().to(device)

x1, x2 = q.forward(st_torch)
sum(p.numel() for p in q.parameters()) # Note: Exactly twice as large as Q-Net in DQN with Keras

453880

In [10]:
p = GaussianPolicy().to(device)



In [11]:
actions = env.get_actions(1)
tmp = x2[actions, actions[0:4]]]

IndexError: too many indices for tensor of dimension 1

In [14]:
import torch.nn.functional as F


F.log_softmax(x2[actions], dim = 0).detach().cpu().numpy()



array([-1.3710837, -1.3732973, -1.4193017, -1.3822424], dtype=float32)

In [24]:
np.random.choice(4, p = F.softmax(x2[actions], dim = 0).detach().cpu().numpy())

3